## 라이브러리 import

In [65]:
import gradio as gr
import requests
import re

############################### GPT 연동 #########################################

def request_gpt (prompt):
    # headers
    headers = {
        "Content-Type" : "application/json",
        "api-key" : api_key
    }

    # body
    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 서울 식당 안내 가이드야. 이모티콘을 추가해서 답변해야해. 참조 데이터 내에서 식당명과 주소를 대답해야 해. 음식점 이름은 name으로 그대로 가져와. 참조를 표시해줘"
            },
            {
                "role": "user",
                "content": "서울 강남에 추천할 맛집이 있을까?"
            },
            {
                "role": "assistant",
                "content": "저는 여기가 맛있을 것 같습니다!:\n난랑\n위치: 서울특별시 강남구 개포동 1163-1\n주소: 서울특별시 강남구 논현로 14\n전화번호: 02-549-2888\n특징: 🇰🇷🇺🇸🇯🇵🇨🇳다국어 메뉴판과 배달 서비스 제공 1 .\n중앙해장\n위치: 서울특별시 강남구 대치동 996-16\n주소: 서울특별시 강남구 영동대로86길 17\n전화번호: 02-558-7905\n특징: 친절한 서비스😁와 우수한 위생 수준✨으로 유명 2 .\n길동원 (대치점)\n위치: 서울특별시 강남구 대치동 1000-6\n주소: 서울특별시 강남구 테헤란로108길 28\n전화번호: 02-553-3384\n특징: 온라인 예약 가능 3 .\n춘천명물닭갈비🐔 (제1호점)\n위치: 서울특별시 강남구 개포동 1231-6\n주소: 서울특별시 강남구 개포로22길 14\n전화번호: 02-575-3308\n특징: 주차 공간 제공 🚗4 .\n구레시시\n위치: 서울특별시 강남구 역삼동 825\n주소: 서울특별시 강남구 강남대로 390\n전화번호: 02-3469-1199\n특징: 친절한 서비스😁와 온라인 예약 가능 💻5 .\n\n이 맛집들은 강남 내에서 다양한 음식을 제공하며, 각각의 특징이 있어 방문하기에 적합한 선택지가 될 수 있습니다. 원하시는 메뉴가 있나요?"
            },
            {
                "role": "user",
                "content": "종로구 맛집 추천해줘"
            },
            {
                "role": "assistant",
                "content": "서울특별시 종로구에서 추천할 만한 맛집은 다음과 같습니다:\n\n송전일식\n종류: 일식🇯🇵\n주소: 서울특별시 종로구 서린동 136\n전화번호: 02-739-8282\n특징: 위생 수준이 우수✨하며 친절한 서비스😁로 잘 알려져 있습니다. 온라인 예약 가능하며 주차 공간과 다국어🇯🇵🇺🇸🇰🇷🇨🇳 메뉴판을 제공합니다 1 .\n나주곰탕 (동대문본점)\n종류: 한식🇯🇵\n주소: 서울특별시 종로구 숭인동 313-8\n전화번호: 02-2236-7006\n특징: 위생 관리✨와 친절한 서비스😁를 자랑하며 온라인 예약💻이 가능합니다 2 .\n두레유\n종류: 한식🇰🇷\n주소: 서울특별시 종로구 평창동 97\n전화번호: 02-3217-1090\n특징: 남녀 화장실이 구분되어 있고 주차 공간🚗 및 무선 인터넷을 제공합니다 3 .\n안래홍\n종류: 중식🇨🇳\n주소: 서울특별시 종로구 공평동 132\n전화번호: 02-739-8013\n특징: 온라인 예약💻이 가능하며 위생 관리✨와 친절한 서비스😁로 알려져 있습니다 4 .\n오두막\n종류: 한식🇰🇷\n주소: 서울특별시 종로구 도렴동 150-6\n전화번호: 02-734-3371\n특징: 광화문 근처에 위치하며 온라인 예약💻이 가능합니다 5 .\n\n이 맛집들은 각각 다양한 메뉴와 분위기를 제공하니 취향에 따라 선택해 보세요!"
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.1,
        "top_p": 0.95,
        "max_tokens": 2000,

        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": ai_search_endpoint,
                    "index_name": ai_search_index,
                    "semantic_configuration": ai_search_semantic,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "in_scope": True,
                    #"role_information": "너는 서울 식당 추천 가이드야",
                    "filter": None,
                    "strictness": 5,
                    "top_n_documents": 5,
                    "authentication": {
                    "type": "api_key",
                    "key": ai_search_api_key,
                    },
                    "key": ai_search_api_key,
                    # "indexName": ai_search_index
                }
            }
        ],
    }
    # API 엔드포인트에 POST 요청을 보내고 응답을 받음
    response = requests.post(endpoint, headers=headers, json=body)
    
    #print(response)
    #print(response.status_code, response.text)

    if response.status_code == 200:

        # 응답을 JSON 형식으로 변환
        response_json = response.json()

        # 응답에서 메시지 및 콘텐츠 필터링 결과 추출
        message = response_json['choices'][0]['message']
        #content_filter = response_json['choices'][0]['content_filter_results']
        citaiton_list = message['context']['citations']

        # 메시지에서 역할(role)과 내용(content) 가져오기
        role = message['role']
        content = message['content']
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)

        #print(role)
        return content ,citaiton_list
    
    else:
        return ""



############################### Gradio #########################################


custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Jua&display=swap');

.custom-container {
    height: 70vh !important;
    background-color : #ffd092;
}
# .custom-textbox {
#     height: 63vh !important;
# }
.send-button{
    height: 10vh !important;
    font-size: 20px !important;
    background-color : #FDEDE1;
}
.custom-title {
    font-family: "Jua", sans-serif !important;  /* Jua 폰트 적용 */
    font-size: 32px !important;  /* 글자 크기 */
    color: #e8947c !important;  /* 글자 색상 */
    text-align: center !important;  /* 중앙 정렬 */
    font-weight: bold !important;  /* 굵은 글씨 */
    margin: 10px !important;  /* 아래 여백 */
}
.custom-title h1 {
    color: #e88064 !important;
}
.custom-tab {
    background-color: #fff7f0 !important;  /* 참조 탭 배경색 */
    border-radius: 10px !important;  /* 모서리 둥글게 */
    padding: 15px !important;  /* 내부 여백 */
    color: #333 !important;  /* 기본 글자색 */
    font-size: 16px !important;  /* 글자 크기 */
    height: 62vh !important;
}

.custom-tab h1, .custom-tab h2, .custom-tab h3 {
    color: #e67e22 !important;  /* 제목 색상 (주황색) */
}

.custom-tab p {
    color: #444 !important;  /* 본문 색상 */
}

.custom-tab-container {
    background-color: #ffe4c4 !important; /* 전체 참조 배경색 */
    padding: 10px !important;
    border-radius: 10px !important;
    box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1) !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    citation_state = gr.State([])
    
    def click_send(prompt, histories):
        print(prompt)
        content, citation_list = request_gpt(prompt=prompt)  # GPT 응답
        
        ## 메세지 구성
        message_list = [{"role": "user", "content": prompt}, {"role": "assistant", "content": content}]
        histories.extend(message_list)
        
        print(histories)
        print(len(citation_list), citation_list)
        return histories, citation_list, "", gr.update(visible=len(citation_list) > 0)
    
    def select_tab(citation):
        print("tab!!!!!!")
        return gr.Textbox(label="", value=citation, lines=20, visible=True)
    
    gr.Markdown("# 🥘🍲서울 맛집! GPT로 찾자!🍔🍜", elem_classes=["custom-title"])
    
    with gr.Row():
        # Chatbot will adjust scale based on whether citations are shown
        chatbot = gr.Chatbot(
            label="서울 맛집 추천", 
            type="messages", 
            scale=6, 
            elem_classes=["custom-container"]
        )
        
        # Citation column - initially hidden
        with gr.Column(scale=1, visible=False) as citation_column:
            @gr.render(inputs=[citation_state])
            def render_citations(citations):
                print(citations)
                if citations:
                    with gr.Tabs():
                        for index, citation in enumerate(citations):
                            key = f"참조{index + 1}"
                            with gr.Tab(key):
                                citation_textbox = gr.Textbox(
                                    label="", 
                                    value=citation['content'], 
                                    lines=20, 
                                    visible=True, elem_classes=["custom-tab"]
                                )
                                citation_textbox.change(
                                    fn=select_tab, 
                                    inputs=[citation_textbox], 
                                    outputs=[citation_textbox]
                                )
    
    with gr.Row():
        prompt_textbox = gr.Textbox(
            label="서울 맛집! 검색해 보세요!", 
            placeholder="대화를 입력하세요.", 
            scale=6
        )
        send_button = gr.Button("전송", scale=1, elem_classes=["send-button"])
    
    send_button.click(
        click_send,
        inputs=[prompt_textbox, chatbot],
        outputs=[chatbot, citation_state, prompt_textbox, citation_column]
    )
    
    prompt_textbox.submit(
        click_send,
        inputs=[prompt_textbox, chatbot],
        outputs=[chatbot, citation_state, prompt_textbox, citation_column]
    )

demo.launch()



* Running on local URL:  http://127.0.0.1:7920

To create a public link, set `share=True` in `launch()`.


[]
[]
강남역 맛집 추천
[{'role': 'user', 'content': '강남역 맛집 추천'}, {'role': 'assistant', 'content': '강남역 근처에서 추천할 맛집은 다음과 같습니다! 😊\n\n1. **빵신 바운스스푼 (강남점)**  \n   - 📍 주소: 서울특별시 강남구 역삼동 832-2  \n   - 🗺️ 상세 위치: 서울특별시 강남구 강남대로 330  \n   - 🍞 특징: 다양한 빵과 디저트를 즐길 수 있는 곳 [참조 1].\n\n2. **더벤티 (강남역지하상가점)**  \n   - 📍 주소: 서울특별시 강남구 역삼동 858  \n   - 🗺️ 상세 위치: 서울특별시 강남구 강남대로 지하 396  \n   - ☕ 특징: 맛있는 음료와 간단한 디저트를 제공하는 카페 [참조 2].\n\n3. **강남육회**  \n   - 📍 주소: 서울특별시 강남구 역삼동 813-16  \n   - 🗺️ 상세 위치: 서울특별시 강남구 봉은사로2길 37  \n   - 🥩 특징: 신선한 육회를 맛볼 수 있는 한식 전문점 [참조 3].\n\n4. **강남장어**  \n   - 📍 주소: 서울특별시 강남구 역삼동 826-9  \n   - 🗺️ 상세 위치: 서울특별시 강남구 테헤란로2길 36  \n   - 🐟 특징: 장어 요리가 유명한 한식 맛집 [참조 4].\n\n5. **강다짐 원유로스페셜티 (강남역)**  \n   - 📍 주소: 서울특별시 강남구 역삼동 858  \n   - 🗺️ 상세 위치: 서울특별시 강남구 강남대로 지하 396  \n   - 🍰 특징: 다양한 디저트와 음료를 즐길 수 있는 카페 [참조 5].\n\n강남역 주변에서 다양한 맛집을 즐겨보세요! 😋'}]
5 [{'content': '일반음식점\n강남점\n중식\n서울특별시 강남구에서 가 볼 만한 식당을 찾으신다면? 강남역 근처에 있는 "뽕신 바운스쿡(강남점)"을 추천합니다!\n37.4926879\n127.0306509\n뽕신 바운스쿡\n서울특별시 강남구 역삼동 832-2\n서울특별시